In [1]:
!pip install strsim

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Libraries required
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from similarity.cosine import Cosine

In [4]:
#Reading and storing the csv file
df = pd.read_csv("Dataset.csv")

In [5]:
#Printing the datset
df

,Course Name,University / Industry Partner Name,Difficulty Level,Course Rating,Course URL,Course Description,all_skill
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...
...,...,...,...,...,...,...,...
4500,Global China: From the Mongols to the Ming,Harvard University,Beginner,NaN,https://www.edx.org/course/global-china-from-t...,Explore the impact of the conquest dynasties a...,"In the 13th century, by force of arms, the Mon..."
4501,Leaders in Citizen Security and Justice Manage...,Inter-American Development Bank,Intermediate,NaN,https://www.edx.org/course/leaders-in-citizen-...,"Learn about the latest in prevention, police a...",The high rates of crime and violence are two o...
4502,Computational Neuroscience: Neuronal Dynamics ...,École polytechnique fédérale de Lausanne,Advanced,NaN,https://www.edx.org/course/computational-neuro...,This course explains the mathematical and comp...,What happens in your brain when you make a dec...
4503,Cities and the Challenge of Sustainable Develo...,SDG Academy,Beginner,NaN,https://www.edx.org/course/cities-and-the-chal...,What is a sustainable city? Learn the basics h...,"According to the United Nations, urbanization ..."


In [7]:
#dropping Nan values
df = df.dropna()

In [8]:
#Function to retrieve a particular difficulty level
def course_difficulty(level):
    data = df.copy().loc[df['Difficulty Level']==level]
    data['index'] = np.arange(0, data.shape[0])
    return data

In [9]:
#Function to clean the data by removing punctutions to increase the efficiency
def remove_punctuations(text):
    text = str(text) 
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [10]:
#Function to find similarity between the course name entered by the user and existing courses in the dataset
def recommender_course(name):
    #Cosine similarity is a metric used to measure how similar the documents
    #Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space.
    cosine = Cosine(2)
    df["name"] = name
    df["p0"] = df["name"].apply(lambda s: cosine.get_profile(s)) 
    df["p1"] = (df["Course Name"]).apply(lambda s: cosine.get_profile(s)) 
    df["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(df["p0"],df["p1"])]
    df.drop(["p0", "p1"], axis=1,inplace =  True)
    return df

In [11]:
#Function to find similarity between the skill entered by the user and existing skills in the dataset
def recommender_system(skill):
     #Cosine similarity is a metric used to measure how similar the documents
    #Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space.
    cosine = Cosine(2)
    data["skill"] = skill
    data["p0"] = data["skill"].apply(lambda s: cosine.get_profile(s)) 
    data["p1"] = data["new_column"].apply(lambda s: cosine.get_profile(s)) 
    data["cosine_sim2"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(data["p0"],data["p1"])]
    data.drop(["p0", "p1"], axis=1,inplace =  True)
    return data

In [13]:
#counter to variable to check whether user wants to give his previous course done 
print("enter Yes to get previous course suggestion as well")
ctr = input()

enter Yes to get previous course suggestion as well
Yes


In [14]:
print(ctr)
if(ctr == "Yes"):
    #if the user gives his previous course then input is taken about it
    print("Enter previous course")
    previous_course = input()
    #The course entered is passed to the function to see similar courses
    final = recommender_course(previous_course)
    final
    #Sorting the dataset in decreasing order of similarity w.r.t the course done by the user
    final = final.sort_values('cosine_sim', ascending = False)
    #Retrieve the top 1000 similar courses
    df = final[0:1000] 
    #Takes the input from user about his level of learning
    print("Enter your level")
    level = input()
    #Level is passsed to the function to retrieve the corresponding courses w.r.t the level
    data = course_difficulty(level)
    data
    #Concating the dataset to make a new column for further analysis
    data['final'] = data['all_skill'] + data['Course Description'] + data['Course Name']
    #Cleaning the dataset
    data['new_column'] = data['final'].apply(remove_punctuations)
    data
    #Taking the skill the user wants to learn
    skill = input('Enter previous skill: ')
    #passing the skill to the function to get similar courses
    final = recommender_system(skill)
    #Sorting the courses in decreasing order of similarity
    data = data.sort_values('cosine_sim2', ascending = False)
else:
    print("Enter your level")
    #Takes the input from user about his level of learning
    level = input()
    #Level is passsed to the function to retrieve the corresponding courses w.r.t the level
    data = course_difficulty(level)
    data
    #Concating the dataset to make a new column for further analysis
    data['final'] = data['all_skill'] + data['Course Description'] + data['Course Name']
    #Cleaning the dataset
    data['new_column'] = data['final'].apply(remove_punctuations)
    #Taking the skill the user wants to learn
    skill = input('Enter previous skill: ')
    #passing the skill to the function to get similar courses
    final = recommender_system(skill)
    #Sorting the courses in decreasing order of similarity
    data = data.sort_values('cosine_sim2', ascending = False)
    

Yes
Enter previous course
Finance for managers
Enter your level
Beginner
Enter previous skill: Finance


In [16]:
#To show the top 5 courses by selecting 5 random from the top 8 similar courses
data[0:8].sample(n=5)

,Course Name,University / Industry Partner Name,Difficulty Level,Course Rating,Course URL,Course Description,all_skill,name,cosine_sim,index,final,new_column,skill,cosine_sim2
2829,Accounting: Principles of Financial Accounting,IESE Business School,Beginner,4.8,https://www.coursera.org/learn/financial-accou...,Financial Accounting is often called the langu...,income statement analysis cash flow Finance...,Finance for managers,0.280945,135,income statement analysis cash flow Finance...,income statement analysis cash flow Finance...,Finance,0.495549
834,FinTech Risk Management,The Hong Kong University of Science and Techno...,Beginner,4.6,https://www.coursera.org/learn/fintech-risk-ma...,This course �FinTech Risk Management� help you...,financial institution Leadership and Manageme...,Finance for managers,0.390360,37,financial institution Leadership and Manageme...,financial institution Leadership and Manageme...,Finance,0.547142
419,Finance for Non-Financial Managers,Emory University,Beginner,4.2,https://www.coursera.org/learn/finance-for-non...,Finance is for �Non-financial Managers� who wa...,balance sheet income statement income prese...,Finance for managers,0.818923,0,balance sheet income statement income prese...,balance sheet income statement income prese...,Finance,0.565221
1164,Corporate Finance Essentials,IESE Business School,Beginner,4.8,https://www.coursera.org/learn/corporate-finan...,Corporate Finance Essentials will enable you t...,Economics Finance investment evaluation Ec...,Finance for managers,0.419961,19,Economics Finance investment evaluation Ec...,Economics Finance investment evaluation Ec...,Finance,0.513063
2068,Principles of Sustainable Finance,Erasmus University Rotterdam,Beginner,4.7,https://www.coursera.org/learn/sustainable-fin...,Finance is widely seen as an obstacle to a bet...,environmental factor corporate governance Fi...,Finance for managers,0.443039,13,environmental factor corporate governance Fi...,environmental factor corporate governance Fi...,Finance,0.526755
